In [1]:
import pandas as pd

# Data Cleaning

In this notebook we will perform the data cleaning from a more engineer point of view. We will check for consistency, analyze possible errors in the data (missing values, outliers) and drop duplicated information

In [2]:
dataset = pd.read_csv('data/raw/amazon_fine_food_reviews.csv')

Missing values in ProfileName and Summary but not in Text and Score that will be our top priority features

In [3]:
dataset.isnull().sum()

Id                         0
ProductId                  0
UserId                     0
ProfileName               16
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64

In [4]:
dataset.Score.value_counts()

5    363122
4     80655
1     52268
3     42640
2     29769
Name: Score, dtype: int64

### Helpfulness review

Helpfulness Numerator: Number of users who found the review helpful

Helpfulness Denominator: Number of users who indicated whether they found the review helpful or not

Considering this, possible feature:
- NotHelpfull instead of the helpfulness denominator
- Percentage of helpfulness

In [5]:
(dataset.HelpfulnessNumerator == dataset.HelpfulnessDenominator).sum() / dataset.shape[0]

0.7975333096433485

In [6]:
(dataset.HelpfulnessDenominator < dataset.HelpfulnessNumerator).sum()

2

In [7]:
dataset[dataset.HelpfulnessDenominator < dataset.HelpfulnessNumerator]

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
44736,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...
64421,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...


We see that there are two rows with inconsistencies in the data since the numerator is greater than the numerator

## Analyzing duplicates

We have considered 3 possible subsets for being a duplicate:

- All columns equal: there is none
- user and text the same: we dropped those which had the same user and text, considering them the same review.
- user and time the same: since the time is in days, it has been discarded because after a brief analysis we discovered that there were genuine different reviews.

For the process of which one keep, we considered to keep the last one and if they are both the same (because after a brief analysis we discovered only a few had different times in the same user and text), we would keep the one with more information in the helfulness columns. But after keeping only the text for the models, we decided to only keep the last that occured. 

One more thing to do for checking the consistency would be check if the score is the same in both reviews.

In [8]:
dataset.duplicated().sum()

0

In [9]:
text_user_duplicated = dataset[dataset.duplicated(subset=['UserId', 'Text'])]

In [10]:
time_user_duplicated = dataset[dataset.duplicated(subset=['UserId', 'Time'])]

### Converting date into datetime

In [11]:
dataset['date'] = pd.to_datetime(dataset['Time'], unit='s')

## Mini-checks of data consistency

## ProfileName and userId?

After analyzing the results, we have dropped the ProfileName. UserId is more consistent for tracking the user, since the ProfileName can change or be anonymous ('A customer'). It could be useful in a very far approach by trying to consider the gender, some demographic information or some analysis in possible change of behaviour when having an open name or not. But it's out of the scope of this analysis.

In [12]:
profname_by_userid = dataset.groupby('UserId').agg({'ProfileName': set})

In [13]:
(profname_by_userid.ProfileName.str.len() == 1).sum() / profname_by_userid.shape[0]

0.9998047325030559

In [14]:
profname_by_userid[profname_by_userid.ProfileName.str.len() != 1].head()

,ProfileName
UserId,
A11CK6P4SLO6RU,"{A Customer, K. Dresser}"
A13K3ZLWAWN1EI,"{A Customer, yogagirl ""grrrl""}"
A16YH487W9ZYO0,"{A Customer, Bruce G. Lindsay}"
A1A0MBT5LKK8U9,"{Lance, A Customer}"
A1CRT2UI0YIOQN,"{A Customer, Krista}"


### Text and summary

The same text doesn't have the same summary, so we have decided to drop the column. This summary isn't very good overall. However, using this or an alternative summary, could be useful maybe if the model that performs the summary + smaller model trained with summaries, is smaller than the current one and it also overcomes the overheat of having two models, then it could be used as an alternative version of the model.

In [15]:
text_by_summary = dataset.groupby('Text').agg({'Summary': set})

Maybe first approach using indistinctly one of them and see if we can get a better summary with one of the AI services

In [16]:
text_by_summary[text_by_summary.Summary.str.len() != 1].head()

,Summary
Text,
"""4C Totally Light"" is one of the very few ""sugar free"" products of it's type to use safer and better tasteing sucralose(Splenda) over Aspertame.<br /><br />Very tasty drink, especially when served over ice!","{4C Totally Light, Better Sweetener!}"
"""By far the best cup of coffee ever!!!"". I received a Keurig machine as a gift for my birthday and had been using the regular run of the mill products that were available. Then low and behold I found the San Francisco Bay brand of one cup brewing pods and you could not pay to stop using them! By far the best cup of coffee ever, every single","{Organic never tasted better!, Looooove It!}"
"""Erythritol"" has become one of our favorite sugar substitutes since I became diabetic a few years ago. My wife uses a little bit of added sugar to bring the sweetness up for the rest of the family in things that need it, but I am happy with it as a one-to-one replacement for sugar in most things.<br /><br />I have not noticed any glycemic impact to my blood sugar or ""laxative"" effects seen with other sugar alcohol sweeteners (which is a very important consideration :-)<br /><br />Highly recommended!<br /><br />CFH",{Great Diabetic Friendly Sweetener - Highly Re...
"""Splenda"" is one of my favorite things ever! Being a Type 2 Diabetic, I am always looking for sweet tasting snacks and drinks that will not effect my blood sugar levels. Splenda makes it possible for us to have sweet baked goods, sweet ice tea and coffee again since I don't like ""Nutra-Sweet"" or ""Sweet and Low""<br /><br />Like most diet products it is not perfect and Splenda does not taste quite as good as regular sugar, but it is as close as we have found. It is really very satisfying and has no calories as well.<br /><br />Highly recommended!",{Great Diabetic Friendly Sweetner! Highly Reco...
"""Swiss Miss No Sugar Added Hot Cocoa"" is a very good Hot Chocolate drink mix! Being a Type 2 Diabetic, I am always looking for sweet tasting snacks and drinks that will not effect my blood sugar levels.<br /><br />Like most diet products, this does not taste quite as good as the regular version, but it is as close as we have found. It is really very satisfying and low in calories as well.<br /><br />Highly recommended!","{Really Good Diet Hot Chocolate! Recommended, ..."


## Function to wrap all the cleaning

Considerations:

- Duplicated rows considered by the same userid and text are dropped
- Id as index
- ProfileName and Summary columns drops
- Since the helpfulness columns are going to be discarded for the model and since they are an insificant number, we will keep the inconsistent rows.

In [17]:
def basic_dataset_cleaning(data):
    duplicated_mask = data.sort_values('Time').duplicated(subset=['UserId', 'Text'], keep='last')
    new_dataset = data.drop(duplicated_mask[duplicated_mask].index, axis=0)
    new_dataset.index = new_dataset['Id']
    new_dataset.drop(['ProfileName', 'Summary', 'Id'], axis=1, inplace=True)
    new_dataset['Time'] = pd.to_datetime(data['Time'], unit='s')
    return new_dataset

In [18]:
new_dataset = basic_dataset_cleaning(dataset)
new_dataset[new_dataset.HelpfulnessDenominator < new_dataset.HelpfulnessNumerator]

,ProductId,UserId,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Text,date
Id,,,,,,,,
44737,B001EQ55RW,A2V0I904FH7ABY,3,2,4,2008-06-05,It was almost a 'love at first bite' - the per...,2008-06-08
64422,B000MIDROQ,A161DK06JJMCYF,3,1,5,2008-08-05,My son loves spaghetti so I didn't hesitate or...,2008-10-25
